In [100]:
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
%run DataUtilsModule.ipynb

In [101]:
class Model(object):
    
    def load_data(self):
        raise NotImplementedError("Load_data() Need To be Implemented")
    
    def create_placeholders(self):
        raise NotImplementedError("create_placeholders() Needs to be Implemented")
        
    def initialize_parameters(self):
        raise NotImplementedError("initialize_parameters() Needs to be Implemented")
        
    def create_feed_dict(self , input_batch , label_batch):
        raise NotImplementedError("create_feed_dict() Needs to be Implemented")
        
    def add_forward_model(self , input_data):
        raise NotImplementedError("add_model() Needs to be Implemented")
        
    def add_cost_op(self , predict):
        raise NotImplementedError("add_loss_op() Needs to be Implemented")
    
    def run_epoch(self , sess , input_data , input_labels):
        raise NotImplementedError("run_epoch Needs to be Implemented")
    
    def fit(self):
        raise NotImplementedError("fit() Needs to be Implemented")
        
    def predict(self , sess , input_data , input_labels=None):
        raise NotImplementedError("predict() Needs to be Implemented")

In [104]:
class cnn(Model):
    def __init__(self):
        tf.set_random_seed(1)
        self.img_height = None
        self.img_width = None
        self.img_channel = None
        self.totalClasses = 5
        self.learning_parameter_shape = {}
        self.learning_parameters = {}
        self.non_learning_parameters = {}
        self.stride = None
        self.window_size = None
        self.gray_scale = False
        self.dropout = []
        self.fully_connented_dims = []
        self.learning_rate = 0.001
        self.num_epochs=50
        self.dataPath = '../data/working/'
        self.minibatch_size = 64
        self.m = None
        #Funtion
        self.load_data()
        self.apply_one_hot()
        self.__set_shape_var_of_Eachlayers()
    
    def apply_one_hot(self):
        self.Y_train = tf.one_hot(self.Y_train , self.totalClasses)
        self.Y_test = tf.one_hot(self.Y_test , self.totalClasses)
        self.Y_dev = tf.one_hot(self.Y_dev , self.totalClasses)
        
        with tf.Session() as sess:
            self.Y_train , self.Y_test,self.Y_dev = sess.run([self.Y_train , self.Y_test , self.Y_dev]) 
        print("After OneHot")
        print(self.X_train.shape)
        print(self.Y_train.shape)

        print(self.X_test.shape)
        print(self.Y_test.shape)

        print(self.X_dev.shape)
        print(self.Y_dev.shape)
            
    def load_data(self):
        #if Data is Already is stored
        if(utils.ifDataExits('X_Train' , self.dataPath)):
            self.X_train , self.X_test , self.X_dev , self.Y_train , self.Y_test , self.Y_dev \
                                                                        = utils.load_saved_data()
            print(self.X_train.shape)
            print(self.Y_train.shape)

            print(self.X_test.shape)
            print(self.Y_test.shape)

            print(self.X_dev.shape)
            print(self.Y_dev.shape)
        else:
            self.X_train,self.X_test,self.Y_train,self.Y_test,self.X_dev,self.Y_dev  = utils.load_data(5 ,img_size=100 , batch_size=400) 
    def __set_shape_var_of_Eachlayers(self):
        #For Learning Parameters
        filter_shape = [
                            [4,4,1,8] ,  #Layer 1
                            [4,4,8,16] , #Layer 2
                            [4,4,16,32] ,#Layer 3 
                            [4,4,32,32] ,#Layer 4
                            [4,4,32,64]  #Layer 5
                        ]
        if not self.gray_scale:
            filter_shape[0][2] = 3 
        
        for index,shape in enumerate(filter_shape):
            layer = index+1
            self.learning_parameter_shape["W" + str(layer)] = shape
        print("Shape :" , self.learning_parameter_shape)
        #For Non-Learning Parameters
        
        #[for first maxpool layer,for second,for third , ...]
        self.window_size = [2,2,2,2,2]
        #[stride of conv layer , stride for max pool layer]
        self.stride = [[1,2] , [1,2] , [1,2] , [1,2] , [1,2]]
        self.dropout = [0 , 0 ,0.3 , 0.4 , 0.4]
        
        #Shape For FullyConnected Layer
        self.fully_connented_dims = [128 , 64 , 5] #for last Layer except sofmax Layer
        
    def __create_placeholders(self):
        X = tf.placeholder(dtype=tf.float32 , 
                           shape=[None , self.img_height , self.img_width , self.img_channel] ,
                           name = 'input_values'
                          )
        Y = tf.placeholder(dtype=tf.float32 ,
                           shape=[None , self.totalClasses] ,
                           name="input_labels"
                          )
        return X,Y
    
    def __initialize_learning_parameters(self):
        layers = len(self.learning_parameter_shape)
        for l in range(1, layers+1):
            self.learning_parameters["W"+str(l)]= \
                                    tf.get_variable("W"+str(l) ,
                                                    self.learning_parameter_shape["W"+str(l)] ,
                                                    initializer=tf.contrib.layers.xavier_initializer(seed=1)
                                                   )
        print("Parameters" , self.learning_parameters)
    def __initialize_non_learning_parameters(self):
        layers = len(self.stride)
        
        for l in range(1,layers + 1):
            self.non_learning_parameters["Conv_s"+str(l)] = self.stride[l-1][0]
            self.non_learning_parameters["Maxpool_s"+str(l)] = self.stride[l-1][1]
            self.non_learning_parameters["Window_size"+str(l)] = self.window_size[l-1]
            self.non_learning_parameters["dropout"+str(l)] = self.dropout[l-1]
            
        
    def __initialize_parameters(self):
        self.__set_shape_var_of_Eachlayers()
        
        self.__initialize_learning_parameters()
        self.__initialize_non_learning_parameters()

        
    def __create_feed_dict(self , input_batch , label_batch):
        feed_dict = {}
        feed_dict[self.X] = input_batch
        feed_dict[self.Y] = label_batch
        
        return feed_dict
    
    def layersBeforeFlatten(self, X):
        LP = self.learning_parameters
        NLP = self.non_learning_parameters
        layers = len(LP)
        
        for l in range(1,layers+1):
            #Extract All parameters Needed
            conv_s = NLP["Conv_s"+str(l)]
            f = NLP["Window_size"+str(l)]
            pool_s = NLP["Maxpool_s"+str(l)]
            curr_filter = LP["W"+str(l)]
            print("curr_filter" , curr_filter)
                
            #CONV Layer
            s = conv_s
            Z = tf.nn.conv2d(X , curr_filter , strides=[1,s,s,1] , padding='SAME')
            
            #Activation
            A = tf.nn.relu(Z)
            
            #Maxpool
            s = pool_s
            features = tf.nn.max_pool(A , ksize=[1,f,f,1] , strides=[1,s,s,1] , padding='SAME')
            X = features
        return features

    def LayersAfterFlatten(self , X):
        layers = len(self.fully_connented_dims)
        
        for l in range(1,layers+1):
            Z = tf.contrib.layers.fully_connected(X  , self.fully_connented_dims[l-1] , activation_fn=None)
        return Z
    
    def __add_forward_model(self , input_data):
        X = self.layersBeforeFlatten(input_data)
        
        X_flatten = tf.contrib.layers.flatten(X)
        
        Z = self.LayersAfterFlatten(X_flatten)
        
        return Z
    
    def add_cost_op(self , predict , Y):
        print("Predict" , np.array(predict).shape)
        print("Cross_entropy {}".format(tf.nn.softmax_cross_entropy_with_logits(logits=predict , labels=Y)))
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict , labels=Y))
        print("Cost {}".format(cost))
        return cost
    
    def get_random_mini_batch(self ,mini_batch_size = 64):
        mini_batches = []
        
        #About Random actually data is already Shuffled

        num_compelete_minibatches = math.floor(self.m / mini_batch_size)
        for k in range(num_compelete_minibatches):
            mini_batch_X = self.X_train[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
            mini_batch_Y = self.Y_train[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
            mini_batch = (mini_batch_X, mini_batch_Y)
            mini_batches.append(mini_batch)
        
        if self.m%num_compelete_minibatches != 0:
            mini_batch_X = self.X_train[num_complete_minibatches * mini_batch_size : m,:,:,:]
            mini_batch_Y = self.Y_train[num_complete_minibatches * mini_batch_size : m,:]
            mini_batch = (mini_batch_X, mini_batch_Y)
            mini_batches.append(mini_batch)

        return mini_batches
    
    def run_epoch(self , sess , input_data , input_label):
        epoch_cost = 0
        minibatches = self.get_random_mini_batch()

        for minibatch in minibatches:
            mini_X , mini_Y = minibatch

            _ , temp_cost , temp_acc= sess.run([self.optimizer , self.cost , self.accuracy] , 
                                     feed_dict = {self.X:mini_X , self.Y:mini_Y})
            print(temp_acc)
            epoch_cost += temp_cost
        
        return epoch_cost 
    
    def add_accuracy(self):
        Xtrain_labels = self.__add_forward_model(tf.cast(self.X_train , "float32"))
        Xtest_labels = self.__add_forward_model(tf.cast(self.X_test , "float32"))
        Xdev_labels = self.__add_forward_model(tf.cast(self.X_dev , "float32"))
        
        correct_prediction_forTrain = tf.equal(tf.argmax(Xtrain_labels,1) , tf.argmax(self.Y_train,1))
        correct_prediction_forTest = tf.equal(tf.argmax(Xtest_labels,1) , tf.argmax(self.Y_test,1))
        correct_prediction_forDev = tf.equal(tf.argmax(Xdev_labels,1) , tf.argmax(self.Y_dev,1))
        
        train_acc = tf.reduce_mean(tf.cast(correct_prediction_forTrain , "float"))
        dev_acc = tf.reduce_mean(tf.cast(correct_prediction_forTest , "float"))
        test_acc = tf.reduce_mean(tf.cast(correct_prediction_forDev , "float"))
        
        return [train_acc , dev_acc , test_acc]

    def fit(self):
        tf.reset_default_graph()
        
        self.m , self.img_height , self.img_width , self.img_channel = self.X_train.shape
        self.totalClasses = self.Y_train.shape[1]
        
        self.X,self.Y = self.__create_placeholders()
        
        self.__initialize_parameters()
        
        Z = self.__add_forward_model(self.X)
        
        self.cost = self.add_cost_op(Z ,self.Y)
        
        self.accuracy = self.add_accuracy()
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
        
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            costs = []
            
            for epoch in range(self.num_epochs):
                epoch_cost  = self.run_epoch(sess , self.X_train , self.Y_train)
                print("Cost after Epoch %i: %f" % (epoch + 1 , epoch_cost) 
#                       "=> Training Acc : %f" %(accuracySet[0]),
#                       "=> Dev Acc : %f" %(accuracySet[1]),
#                       "=> Test Acc : %f" %(accuracySet[2])
                     )
                costs.append(epoch_cost)
        
        plt.plot(range(1,self.num_epochs+1) , costs)
        plt.ylabel('cost')
        plt.xlabel('iteration')
        plt.show()